Instalamos ultralytics y roboflow desde terminal:
- pip install ultralytics
- pip install roboflow

# Solutions ParkingPtsSelection Ultralytics - Modelo VisDrone YOLOv11s

Vamos a entrenar un modelo VisDrone YOLOv11s para la detección de vehículos desde vista de águila

Abrimos la terminal y ejecutamos lo siguiente:
```
python
```
```
from ultralytics import solutions
```
```
solutions.ParkingPtsSelection()
```
Como en MacOS no funciona correctamente debido al fallo en una clase para leer los formatos de imagen, se procede a modificarla manualmente y crear un fichero llamado "seleccionador_secciones.ipynb" que simula con Tkinter lo que ejecutaríamos desde el código anterior. Ahora ya podemos seleccionar una imagen para indicar donde se encuentras las plazas de parking disponibles.

### Después del archivo JSON

Una vez hemos obtenido el archivo JSON, tenemos que recorrer en un for para colocar todos los rectángulos de reconocimiento de plazas de garaje. Lo que hace este modelo es identificar cuando un coche está dentro de cada rectángulo que indica la plaza de parking y así mostrarlo en rojo (ocupado) o verde (libre). Se hace una demostración a tiempo real y también se guarda un fichero con el video.

## Parking 3 - Alta calidad de imagen

### Plazas con rectángulos

In [1]:
###### PLAZAS OCUPADAS CON RECTÁNGULOS

import json
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont  # Importamos PIL para usar la fuente personalizada

# Cargar coordenadas de las plazas desde el JSON
with open('JSON/bounding_boxes_parking3.json', 'r') as file:
    parking_coordinates = json.load(file)  # Contiene las plazas como listas de puntos

def is_point_inside_polygon(point, polygon):
    """Comprueba si un punto (x, y) está dentro del polígono definido por las plazas."""
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def check_occupancy(detections, parking_coordinates):
    """Comprueba si las detecciones ocupan alguna plaza."""
    for park in parking_coordinates:
        park['occupied'] = False  # Reiniciar el estado de ocupación
        for detection in detections:
            det_x1, det_y1, det_x2, det_y2 = detection['bbox']
            vehicle_center = ((det_x1 + det_x2) / 2, (det_y1 + det_y2) / 2)  # Centro del vehículo

            # Verificar si el centro del vehículo está dentro del polígono de la plaza
            if is_point_inside_polygon(vehicle_center, park['points']):
                park['occupied'] = True
                break
    return parking_coordinates

def draw_frame_with_parking_status(frame, parking_coordinates, background_color=(255, 255, 255), font_path="fuentes/Parkinsans-VariableFont_wght.ttf"):
    """Dibuja el estado del parking en el frame, incluyendo un fondo a la derecha para las plazas libres."""
    free_count = 0
    occupied_count = 0
    free_spaces = []  # Lista de plazas libres

    # Dibujar las plazas numeradas y su estado
    for idx, park in enumerate(parking_coordinates):
        points = np.array(park['points'])
        x1, y1 = points[:, 0].min(), points[:, 1].min()
        x2, y2 = points[:, 0].max(), points[:, 1].max()

        color = (0, 0, 255) if park['occupied'] else (0, 255, 0)  # Rojo para ocupada, verde para libre
        if park['occupied']:
            occupied_count += 1
        else:
            free_count += 1
            free_spaces.append(idx + 1)  # Guardar el número de plaza libre (1-indexado)

        # Dibujar rectángulo
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Borde con grosor de 2

        # Agregar número de plaza en la esquina superior izquierda
        cv2.putText(frame, f'{idx + 1}', (x1 + 5, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Crear el fondo adicional (a la derecha)
    frame_height, frame_width = frame.shape[:2]
    new_width = frame_width + 300  # Añadir 300 píxeles para el fondo
    new_frame = np.ones((frame_height, new_width, 3), dtype=np.uint8) * np.array(background_color, dtype=np.uint8)

    # Copiar el video original al nuevo frame
    new_frame[:, :frame_width] = frame

    # Crear un área para mostrar las estadísticas en la parte superior de la columna
    y_offset = 30  # Espacio para las estadísticas arriba de las plazas libres
    start_x = frame_width + 20  # Comienza justo después del video original

    # Usar PIL para cargar la fuente
    pil_image = Image.fromarray(new_frame)
    draw = ImageDraw.Draw(pil_image)

    try:
        font = ImageFont.truetype(font_path, 20)  # Ajusta el tamaño de la fuente si es necesario
    except IOError:
        print("No se pudo cargar la fuente. Asegúrate de tener el archivo .ttf correctamente ubicado.")
        font = ImageFont.load_default()  # Cargar fuente por defecto si no se encuentra

    # Escribir estadísticas de plazas libres y ocupadas
    total_spaces = len(parking_coordinates)
    occupancy_rate = (occupied_count / total_spaces) * 100 if total_spaces > 0 else 0

    # Mostrar la tasa de ocupación
    draw.text((start_x, y_offset), f'Tasa de ocupación: {occupancy_rate:.1f}%', font=font, fill=(0, 0, 0))
    y_offset += 40  # Espacio para separar del listado

    # Mostrar plazas libres y ocupadas
    draw.text((start_x, y_offset), f'Plazas ocupadas: {occupied_count}', font=font, fill=(0, 0, 255))
    y_offset += 40  # Espacio entre las estadísticas
    draw.text((start_x, y_offset), f'Plazas libres: {free_count}', font=font, fill=(0, 255, 0))
    y_offset += 40  # Espacio para la tasa de ocupación

    # Listar las plazas libres
    for idx, space in enumerate(free_spaces):
        draw.text((start_x, y_offset), f'Plaza {space}', font=font, fill=(0, 0, 0))
        y_offset += 30  # Separar cada línea

    # Convertir la imagen de nuevo a un array de OpenCV
    new_frame = np.array(pil_image)

    return new_frame

def process_video(input_video_path, output_video_path, parking_coordinates, frame_skip=100):
    """Procesa un video, analizando plaza por plaza y genera un nuevo video."""
    model = YOLO("Modelos/visdrone_yolov11s.pt")  # Ruta a tu modelo entrenado

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el archivo de entrada {input_video_path}.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if frame_width == 0 or frame_height == 0:
        print("Error: Dimensiones del video de entrada no válidas.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Cambiar a un codec compatible
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (frame_width + 300, frame_height))  # +300 para el fondo

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    parking_status = parking_coordinates

    try:
        while cap.isOpened() and frame_count < total_frames:
            ret, frame = cap.read()
            if not ret or frame is None:
                print(f"Advertencia: No se pudo leer el frame {frame_count}.")
                break

            if frame_count % frame_skip == 0:
                results = model(frame)
                detections = []
                for result in results:
                    for box in result.boxes.xyxy.cpu().numpy():
                        x1, y1, x2, y2 = box
                        detections.append({'bbox': [x1, y1, x2, y2]})
                parking_status = check_occupancy(detections, parking_coordinates)

            frame = draw_frame_with_parking_status(frame, parking_status)
            if frame is None or frame.size == 0:
                print(f"Error: El frame procesado no es válido.")
                break

            out.write(frame)
            print(f"Frame {frame_count} procesado y escrito al archivo de salida.")

            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            frame_count += 1

    except Exception as e:
        print(f"Error inesperado: {e}")

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Rutas de los archivos
input_video_path = "Videos/video_parking3.mp4"  # Cambia este por tu video de entrada
output_video_path = "parking_management_processed.mp4"

# Ruta de la fuente personalizada
font_path = "fuentes/Parkinsans-VariableFont_wght.ttf"  # Ajusta esta ruta si es necesario

# Procesar el video
process_video(input_video_path, output_video_path, parking_coordinates, frame_skip=100)


0: 448x640 1 pedestrian, 56 cars, 3 vans, 2 trucks, 269.9ms
Speed: 5.7ms preprocess, 269.9ms inference, 2.4ms postprocess per image at shape (1, 3, 448, 640)
Frame 0 procesado y escrito al archivo de salida.
Frame 1 procesado y escrito al archivo de salida.
Frame 2 procesado y escrito al archivo de salida.
Frame 3 procesado y escrito al archivo de salida.
Frame 4 procesado y escrito al archivo de salida.
Frame 5 procesado y escrito al archivo de salida.
Frame 6 procesado y escrito al archivo de salida.
Frame 7 procesado y escrito al archivo de salida.
Frame 8 procesado y escrito al archivo de salida.


2024-11-26 12:58:24.504 python[69659:1815504] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-26 12:58:24.504 python[69659:1815504] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Frame 9 procesado y escrito al archivo de salida.
Frame 10 procesado y escrito al archivo de salida.
Frame 11 procesado y escrito al archivo de salida.
Frame 12 procesado y escrito al archivo de salida.
Frame 13 procesado y escrito al archivo de salida.
Frame 14 procesado y escrito al archivo de salida.
Frame 15 procesado y escrito al archivo de salida.
Frame 16 procesado y escrito al archivo de salida.
Frame 17 procesado y escrito al archivo de salida.
Frame 18 procesado y escrito al archivo de salida.
Frame 19 procesado y escrito al archivo de salida.
Frame 20 procesado y escrito al archivo de salida.
Frame 21 procesado y escrito al archivo de salida.
Frame 22 procesado y escrito al archivo de salida.
Frame 23 procesado y escrito al archivo de salida.
Frame 24 procesado y escrito al archivo de salida.
Frame 25 procesado y escrito al archivo de salida.
Frame 26 procesado y escrito al archivo de salida.
Frame 27 procesado y escrito al archivo de salida.
Frame 28 procesado y escrito al 

2024-11-26 12:58:34.858 python[69659:1815504] WARNING: -finishWriting should not be called on the main thread.


KeyboardInterrupt: 

### Plazas con puntos

In [1]:
####### PLAZAS OCUPADAS CON PUNTOS

import json
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont  # Importamos PIL para usar la fuente personalizada

# Cargar coordenadas de las plazas desde el JSON
with open('JSON/bounding_boxes_parking3.json', 'r') as file:
    parking_coordinates = json.load(file)  # Contiene las plazas como listas de puntos

def is_point_inside_polygon(point, polygon):
    """Comprueba si un punto (x, y) está dentro del polígono definido por las plazas."""
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def check_occupancy(detections, parking_coordinates):
    """Comprueba si las detecciones ocupan alguna plaza."""
    for park in parking_coordinates:
        park['occupied'] = False  # Reiniciar el estado de ocupación
        for detection in detections:
            det_x1, det_y1, det_x2, det_y2 = detection['bbox']
            vehicle_center = ((det_x1 + det_x2) / 2, (det_y1 + det_y2) / 2)  # Centro del vehículo

            # Verificar si el centro del vehículo está dentro del polígono de la plaza
            if is_point_inside_polygon(vehicle_center, park['points']):
                park['occupied'] = True
                break
    return parking_coordinates

def draw_frame_with_parking_status(frame, parking_coordinates, background_color=(255, 255, 255), font_path="fuentes/Parkinsans-VariableFont_wght.ttf"):
    """Dibuja el estado del parking en el frame, incluyendo un fondo a la derecha para las plazas libres."""
    free_count = 0
    occupied_count = 0
    free_spaces = []  # Lista de plazas libres

    # Dibujar las plazas numeradas y su estado
    for idx, park in enumerate(parking_coordinates):
        points = np.array(park['points'])
        x1, y1 = points[:, 0].min(), points[:, 1].min()
        x2, y2 = points[:, 0].max(), points[:, 1].max()

        color = (0, 0, 255) if park['occupied'] else (0, 255, 0)  # Rojo para ocupada, verde para libre
        if park['occupied']:
            occupied_count += 1
        else:
            free_count += 1
            free_spaces.append(idx + 1)  # Guardar el número de plaza libre (1-indexado)

        # Dibujar rectángulo
        #cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Borde con grosor de 2

        # Dibujar el indicador de ocupación (círculo)
        indicator_color = (0, 255, 0) if not park['occupied'] else (0, 0, 255)  # Verde si libre, rojo si ocupado
        cv2.circle(frame, (x1 + 10, y1 + 10), 8, indicator_color, -1)  # Círculo sólido

        # Agregar número de plaza en la esquina superior izquierda
        cv2.putText(frame, f'{idx + 1}', (x1 + 20, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Crear el fondo adicional (a la derecha)
    frame_height, frame_width = frame.shape[:2]
    new_width = frame_width + 300  # Añadir 300 píxeles para el fondo
    new_frame = np.ones((frame_height, new_width, 3), dtype=np.uint8) * np.array(background_color, dtype=np.uint8)

    # Copiar el video original al nuevo frame
    new_frame[:, :frame_width] = frame

    # Crear un área para mostrar las estadísticas en la parte superior de la columna
    y_offset = 30  # Espacio para las estadísticas arriba de las plazas libres
    start_x = frame_width + 20  # Comienza justo después del video original

    # Usar PIL para cargar la fuente
    pil_image = Image.fromarray(new_frame)
    draw = ImageDraw.Draw(pil_image)

    try:
        font = ImageFont.truetype(font_path, 20)  # Ajusta el tamaño de la fuente si es necesario
    except IOError:
        print("No se pudo cargar la fuente. Asegúrate de tener el archivo .ttf correctamente ubicado.")
        font = ImageFont.load_default()  # Cargar fuente por defecto si no se encuentra

    # Escribir estadísticas de plazas libres y ocupadas
    total_spaces = len(parking_coordinates)
    occupancy_rate = (occupied_count / total_spaces) * 100 if total_spaces > 0 else 0

    # Mostrar la tasa de ocupación
    draw.text((start_x, y_offset), f'Tasa de ocupación: {occupancy_rate:.1f}%', font=font, fill=(0, 0, 0))
    y_offset += 40  # Espacio para separar del listado

    # Mostrar plazas libres y ocupadas
    draw.text((start_x, y_offset), f'Plazas ocupadas: {occupied_count}', font=font, fill=(0, 0, 255))
    y_offset += 40  # Espacio entre las estadísticas
    draw.text((start_x, y_offset), f'Plazas libres: {free_count}', font=font, fill=(0, 255, 0))
    y_offset += 40  # Espacio para la tasa de ocupación

    # Listar las plazas libres
    for idx, space in enumerate(free_spaces):
        draw.text((start_x, y_offset), f'Plaza {space}', font=font, fill=(0, 0, 0))
        y_offset += 30  # Separar cada línea

    # Convertir la imagen de nuevo a un array de OpenCV
    new_frame = np.array(pil_image)

    return new_frame

def process_video(input_video_path, output_video_path, parking_coordinates, frame_skip=100):
    """Procesa un video, analizando plaza por plaza y genera un nuevo video."""
    model = YOLO("Modelos/visdrone_yolov11s.pt")  # Ruta a tu modelo entrenado

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el archivo de entrada {input_video_path}.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if frame_width == 0 or frame_height == 0:
        print("Error: Dimensiones del video de entrada no válidas.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Cambiar a un codec compatible
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (frame_width + 300, frame_height))  # +300 para el fondo

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    parking_status = parking_coordinates

    try:
        while cap.isOpened() and frame_count < total_frames:
            ret, frame = cap.read()
            if not ret or frame is None:
                print(f"Advertencia: No se pudo leer el frame {frame_count}.")
                break

            if frame_count % frame_skip == 0:
                results = model(frame)
                detections = []
                for result in results:
                    for box in result.boxes.xyxy.cpu().numpy():
                        x1, y1, x2, y2 = box
                        detections.append({'bbox': [x1, y1, x2, y2]})
                parking_status = check_occupancy(detections, parking_coordinates)

            frame = draw_frame_with_parking_status(frame, parking_status)
            if frame is None or frame.size == 0:
                print(f"Error: El frame procesado no es válido.")
                break

            out.write(frame)
            print(f"Frame {frame_count} procesado y escrito al archivo de salida.")

            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            frame_count += 1

    except Exception as e:
        print(f"Error inesperado: {e}")

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Rutas de los archivos
input_video_path = "Videos/video_parking3.mp4"  # Cambia este por tu video de entrada
output_video_path = "parking_management_processed.mp4"

# Ruta de la fuente personalizada
font_path = "fuentes/Parkinsans-VariableFont_wght.ttf"  # Ajusta esta ruta si es necesario

# Procesar el video
process_video(input_video_path, output_video_path, parking_coordinates, frame_skip=100)


0: 448x640 1 pedestrian, 56 cars, 3 vans, 2 trucks, 301.3ms
Speed: 9.9ms preprocess, 301.3ms inference, 16.3ms postprocess per image at shape (1, 3, 448, 640)
Frame 0 procesado y escrito al archivo de salida.
Frame 1 procesado y escrito al archivo de salida.
Frame 2 procesado y escrito al archivo de salida.
Frame 3 procesado y escrito al archivo de salida.
Frame 4 procesado y escrito al archivo de salida.
Frame 5 procesado y escrito al archivo de salida.
Frame 6 procesado y escrito al archivo de salida.
Frame 7 procesado y escrito al archivo de salida.
Frame 8 procesado y escrito al archivo de salida.


2024-11-26 13:07:07.366 python[1069:14639] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-26 13:07:07.366 python[1069:14639] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Frame 9 procesado y escrito al archivo de salida.
Frame 10 procesado y escrito al archivo de salida.
Frame 11 procesado y escrito al archivo de salida.
Frame 12 procesado y escrito al archivo de salida.
Frame 13 procesado y escrito al archivo de salida.
Frame 14 procesado y escrito al archivo de salida.
Frame 15 procesado y escrito al archivo de salida.
Frame 16 procesado y escrito al archivo de salida.
Frame 17 procesado y escrito al archivo de salida.
Frame 18 procesado y escrito al archivo de salida.
Frame 19 procesado y escrito al archivo de salida.
Frame 20 procesado y escrito al archivo de salida.
Frame 21 procesado y escrito al archivo de salida.
Frame 22 procesado y escrito al archivo de salida.
Frame 23 procesado y escrito al archivo de salida.
Frame 24 procesado y escrito al archivo de salida.
Frame 25 procesado y escrito al archivo de salida.
Frame 26 procesado y escrito al archivo de salida.
Frame 27 procesado y escrito al archivo de salida.
Frame 28 procesado y escrito al 

2024-11-26 13:07:40.339 python[1069:14639] WARNING: -finishWriting should not be called on the main thread.


## Parking 2

### Redimensión de video

El video "video_parking2_recortado.mp4" tiene unas dimensiones iniciales de 432 x 290 píxeles. El anterior video, "video_parking3.mp4" con el que hemos trabajado las tenía de 1100 x 720 píxeles. Cuanto más grande sea el video más accuracy tendrá nuestro programa ya que será más fácil detectar los vehículos. 
Con CV2 vamos a redimensionar frame a frame el video a un tamaño más grande y, con el nuevo video redimensionado, procederemos a aplicar la IA de detección de plazas.

In [2]:
import cv2

def resize_video(input_video_path, output_video_path, new_width=1000, new_height=671):
    """Redimensiona el video de entrada y lo guarda como un nuevo video."""
    # Abrir el video de entrada
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el archivo de entrada {input_video_path}.")
        return

    # Obtener las dimensiones originales del video
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Definir el codec y crear el objeto VideoWriter para guardar el video redimensionado
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Codec MJPG
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (new_width, new_height))  # Tamaño nuevo

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensionar el frame
        resized_frame = cv2.resize(frame, (new_width, new_height))

        # Escribir el frame redimensionado en el archivo de salida
        out.write(resized_frame)

        frame_count += 1

    # Liberar los recursos
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"El video ha sido redimensionado y guardado como {output_video_path}. Total de frames procesados: {frame_count}")

# Rutas de los archivos de entrada y salida
input_video_path = "Videos/video_parking2_recortado.mp4"  # Cambia este por tu video de entrada
output_video_path = "Videos/video_parking2_resized.mp4"  # Cambia este por el archivo de salida

# Redimensionar el video
resize_video(input_video_path, output_video_path, new_width=1000, new_height=671)

OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write vide

El video ha sido redimensionado y guardado como Videos/video_parking2_resized.mp4. Total de frames procesados: 2207


OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write vide

### Plazas con rectángulos

In [1]:
import json
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont  # Importamos PIL para usar la fuente personalizada

# Cargar coordenadas de las plazas desde el JSON
with open('JSON/bounding_boxes_parking2.json', 'r') as file:
    parking_coordinates = json.load(file)  # Contiene las plazas como listas de puntos

def is_point_inside_polygon(point, polygon):
    """Comprueba si un punto (x, y) está dentro del polígono definido por las plazas."""
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def check_occupancy(detections, parking_coordinates):
    """Comprueba si las detecciones ocupan alguna plaza."""
    for park in parking_coordinates:
        park['occupied'] = False  # Reiniciar el estado de ocupación
        for detection in detections:
            det_x1, det_y1, det_x2, det_y2 = detection['bbox']
            vehicle_center = ((det_x1 + det_x2) / 2, (det_y1 + det_y2) / 2)  # Centro del vehículo

            # Verificar si el centro del vehículo está dentro del polígono de la plaza
            if is_point_inside_polygon(vehicle_center, park['points']):
                park['occupied'] = True
                break
    return parking_coordinates

def draw_frame_with_parking_status(frame, parking_coordinates, background_color=(255, 255, 255), font_path="fuentes/Parkinsans-VariableFont_wght.ttf"):
    """Dibuja el estado del parking en el frame, incluyendo un fondo a la derecha para las plazas libres."""
    free_count = 0
    occupied_count = 0
    free_spaces = []  # Lista de plazas libres

    # Dibujar las plazas numeradas y su estado
    for idx, park in enumerate(parking_coordinates):
        points = np.array(park['points'])
        x1, y1 = points[:, 0].min(), points[:, 1].min()
        x2, y2 = points[:, 0].max(), points[:, 1].max()

        color = (0, 0, 255) if park['occupied'] else (0, 255, 0)  # Rojo para ocupada, verde para libre
        if park['occupied']:
            occupied_count += 1
        else:
            free_count += 1
            free_spaces.append(idx + 1)  # Guardar el número de plaza libre (1-indexado)

        # Dibujar rectángulo
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Borde con grosor de 2

        # Agregar número de plaza en la esquina superior izquierda
        cv2.putText(frame, f'{idx + 1}', (x1 + 5, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Crear el fondo adicional (a la derecha)
    frame_height, frame_width = frame.shape[:2]
    new_width = frame_width + 300  # Añadir 300 píxeles para el fondo
    new_frame = np.ones((frame_height, new_width, 3), dtype=np.uint8) * np.array(background_color, dtype=np.uint8)

    # Copiar el video original al nuevo frame
    new_frame[:, :frame_width] = frame

    # Crear un área para mostrar las estadísticas en la parte superior de la columna
    y_offset = 30  # Espacio para las estadísticas arriba de las plazas libres
    start_x = frame_width + 20  # Comienza justo después del video original

    # Usar PIL para cargar la fuente
    pil_image = Image.fromarray(new_frame)
    draw = ImageDraw.Draw(pil_image)

    try:
        font = ImageFont.truetype(font_path, 20)  # Ajusta el tamaño de la fuente si es necesario
    except IOError:
        print("No se pudo cargar la fuente. Asegúrate de tener el archivo .ttf correctamente ubicado.")
        font = ImageFont.load_default()  # Cargar fuente por defecto si no se encuentra

    # Escribir estadísticas de plazas libres y ocupadas
    total_spaces = len(parking_coordinates)
    occupancy_rate = (occupied_count / total_spaces) * 100 if total_spaces > 0 else 0

    # Mostrar la tasa de ocupación
    draw.text((start_x, y_offset), f'Tasa de ocupación: {occupancy_rate:.1f}%', font=font, fill=(0, 0, 0))
    y_offset += 40  # Espacio para separar del listado

    # Mostrar plazas libres y ocupadas
    draw.text((start_x, y_offset), f'Plazas ocupadas: {occupied_count}', font=font, fill=(0, 0, 255))
    y_offset += 40  # Espacio entre las estadísticas
    draw.text((start_x, y_offset), f'Plazas libres: {free_count}', font=font, fill=(0, 255, 0))
    y_offset += 40  # Espacio para la tasa de ocupación

    # Listar las plazas libres
    for idx, space in enumerate(free_spaces):
        draw.text((start_x, y_offset), f'Plaza {space}', font=font, fill=(0, 0, 0))
        y_offset += 30  # Separar cada línea

    # Convertir la imagen de nuevo a un array de OpenCV
    new_frame = np.array(pil_image)

    return new_frame

def process_video(input_video_path, output_video_path, parking_coordinates, frame_skip=50):
    """Procesa un video, analizando plaza por plaza y genera un nuevo video."""
    model = YOLO("Modelos/visdrone_yolov11s.pt")  # Ruta a tu modelo entrenado

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el archivo de entrada {input_video_path}.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if frame_width == 0 or frame_height == 0:
        print("Error: Dimensiones del video de entrada no válidas.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Cambiar a un codec compatible
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (frame_width + 300, frame_height))  # +300 para el fondo

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    parking_status = parking_coordinates

    try:
        while cap.isOpened() and frame_count < total_frames:
            ret, frame = cap.read()
            if not ret or frame is None:
                print(f"Advertencia: No se pudo leer el frame {frame_count}.")
                break

            if frame_count % frame_skip == 0:
                results = model(frame)
                detections = []
                for result in results:
                    for box in result.boxes.xyxy.cpu().numpy():
                        x1, y1, x2, y2 = box
                        detections.append({'bbox': [x1, y1, x2, y2]})
                parking_status = check_occupancy(detections, parking_coordinates)

            frame = draw_frame_with_parking_status(frame, parking_status)
            if frame is None or frame.size == 0:
                print(f"Error: El frame procesado no es válido.")
                break

            out.write(frame)
            print(f"Frame {frame_count} procesado y escrito al archivo de salida.")

            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            frame_count += 1

    except Exception as e:
        print(f"Error inesperado: {e}")

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Rutas de los archivos
input_video_path = "Videos/video_parking2_resized.mp4"  # Cambia este por tu video de entrada
output_video_path = "parking_management_processed.mp4"

# Ruta de la fuente personalizada
font_path = "fuentes/Parkinsans-VariableFont_wght.ttf"  # Ajusta esta ruta si es necesario

# Procesar el video
process_video(input_video_path, output_video_path, parking_coordinates, frame_skip=50)


0: 448x640 46 cars, 45 vans, 2 trucks, 1 bus, 221.0ms
Speed: 6.4ms preprocess, 221.0ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Frame 0 procesado y escrito al archivo de salida.
Frame 1 procesado y escrito al archivo de salida.
Frame 2 procesado y escrito al archivo de salida.
Frame 3 procesado y escrito al archivo de salida.
Frame 4 procesado y escrito al archivo de salida.
Frame 5 procesado y escrito al archivo de salida.
Frame 6 procesado y escrito al archivo de salida.
Frame 7 procesado y escrito al archivo de salida.
Frame 8 procesado y escrito al archivo de salida.
Frame 9 procesado y escrito al archivo de salida.
Frame 10 procesado y escrito al archivo de salida.


2024-11-26 12:55:47.491 python[69626:1811293] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-26 12:55:47.491 python[69626:1811293] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Frame 11 procesado y escrito al archivo de salida.
Frame 12 procesado y escrito al archivo de salida.
Frame 13 procesado y escrito al archivo de salida.
Frame 14 procesado y escrito al archivo de salida.
Frame 15 procesado y escrito al archivo de salida.
Frame 16 procesado y escrito al archivo de salida.
Frame 17 procesado y escrito al archivo de salida.
Frame 18 procesado y escrito al archivo de salida.
Frame 19 procesado y escrito al archivo de salida.
Frame 20 procesado y escrito al archivo de salida.
Frame 21 procesado y escrito al archivo de salida.
Frame 22 procesado y escrito al archivo de salida.
Frame 23 procesado y escrito al archivo de salida.
Frame 24 procesado y escrito al archivo de salida.
Frame 25 procesado y escrito al archivo de salida.
Frame 26 procesado y escrito al archivo de salida.
Frame 27 procesado y escrito al archivo de salida.
Frame 28 procesado y escrito al archivo de salida.
Frame 29 procesado y escrito al archivo de salida.
Frame 30 procesado y escrito al

2024-11-26 12:56:01.141 python[69626:1811293] WARNING: -finishWriting should not be called on the main thread.

KeyboardInterrupt



## Gradio

In [6]:
## INTERFAZ QUE PERMITE SELECCIONAR RECTÁNGULOS
import gradio as gr
from gradio_image_prompter import ImagePrompter

demo = gr.Interface(
    lambda prompts: (prompts["image"], prompts["points"]),
    ImagePrompter(show_label=False),
    [gr.Image(show_label=False), gr.Dataframe(label="Points")],
)
demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [19]:
### CÓDIGO QUE CREA JSON Y PROCESA EL VIDEO

import pandas as pd
import json
import gradio as gr
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import uuid
import os

# Función para procesar el CSV y generar el JSON
def create_json_from_csv(csv_file):
    # Leer el archivo CSV
    data = pd.read_csv(csv_file)
    rectangles = []

    # Procesar las filas del DataFrame
    for index, row in data.iterrows():
        try:
            # Parsear la columna "Points" como JSON
            points_dict = json.loads(row["Points"])
            raw_coordinates = points_dict["data"]

            # Procesar cada conjunto de coordenadas
            for rect in raw_coordinates:
                x1, y1 = rect[0], rect[1]  # Esquina superior izquierda
                x2, y2 = rect[3], rect[4]  # Esquina inferior derecha

                # Crear el rectángulo
                rectangle = {
                    "points": [
                        [x1, y1],
                        [x2, y1],
                        [x2, y2],
                        [x1, y2]
                    ]
                }
                rectangles.append(rectangle)
        except Exception as e:
            return f"Error procesando la fila {index}: {e}"

    # Convertir la lista de rectángulos en JSON
    rectangles_json = json.dumps(rectangles, indent=4)

    # Guardar el JSON en un archivo
    output_file = "rectangles.json"
    with open(output_file, "w") as f:
        f.write(rectangles_json)

def is_point_inside_polygon(point, polygon):
    """Comprueba si un punto (x, y) está dentro del polígono definido por las plazas."""
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def check_occupancy(detections, parking_coordinates):
    """Comprueba si las detecciones ocupan alguna plaza."""
    for park in parking_coordinates:
        park['occupied'] = False  # Reiniciar el estado de ocupación
        for detection in detections:
            det_x1, det_y1, det_x2, det_y2 = detection['bbox']
            vehicle_center = ((det_x1 + det_x2) / 2, (det_y1 + det_y2) / 2)  # Centro del vehículo

            # Verificar si el centro del vehículo está dentro del polígono de la plaza
            if is_point_inside_polygon(vehicle_center, park['points']):
                park['occupied'] = True
                break
    return parking_coordinates

def draw_frame_with_parking_status(frame, parking_coordinates, background_color=(255, 255, 255), font_path="fuentes/Parkinsans-VariableFont_wght.ttf"):
    """Dibuja el estado del parking en el frame, incluyendo un fondo a la derecha para las plazas libres."""
    free_count = 0
    occupied_count = 0
    free_spaces = []  # Lista de plazas libres

    # Dibujar las plazas numeradas y su estado
    for idx, park in enumerate(parking_coordinates):
        points = np.array(park['points'])
        x1, y1 = points[:, 0].min(), points[:, 1].min()
        x2, y2 = points[:, 0].max(), points[:, 1].max()

        color = (0, 0, 255) if park['occupied'] else (0, 255, 0)  # Rojo para ocupada, verde para libre
        if park['occupied']:
            occupied_count += 1
        else:
            free_count += 1
            free_spaces.append(idx + 1)  # Guardar el número de plaza libre (1-indexado)

        # Dibujar rectángulo
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Borde con grosor de 2

        # Agregar número de plaza en la esquina superior izquierda
        cv2.putText(frame, f'{idx + 1}', (x1 + 5, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Crear el fondo adicional (a la derecha)
    frame_height, frame_width = frame.shape[:2]
    new_width = frame_width + 300  # Añadir 300 píxeles para el fondo
    new_frame = np.ones((frame_height, new_width, 3), dtype=np.uint8) * np.array(background_color, dtype=np.uint8)

    # Copiar el video original al nuevo frame
    new_frame[:, :frame_width] = frame

    # Crear un área para mostrar las estadísticas en la parte superior de la columna
    y_offset = 30  # Espacio para las estadísticas arriba de las plazas libres
    start_x = frame_width + 20  # Comienza justo después del video original

    # Usar PIL para cargar la fuente
    pil_image = Image.fromarray(new_frame)
    draw = ImageDraw.Draw(pil_image)

    try:
        font = ImageFont.truetype(font_path, 20)  # Ajusta el tamaño de la fuente si es necesario
    except IOError:
        print("No se pudo cargar la fuente. Asegúrate de tener el archivo .ttf correctamente ubicado.")
        font = ImageFont.load_default()  # Cargar fuente por defecto si no se encuentra

    # Escribir estadísticas de plazas libres y ocupadas
    total_spaces = len(parking_coordinates)
    occupancy_rate = (occupied_count / total_spaces) * 100 if total_spaces > 0 else 0

    # Mostrar la tasa de ocupación
    draw.text((start_x, y_offset), f'Tasa de ocupación: {occupancy_rate:.1f}%', font=font, fill=(0, 0, 0))
    y_offset += 40  # Espacio para separar del listado

    # Mostrar plazas libres y ocupadas
    draw.text((start_x, y_offset), f'Plazas ocupadas: {occupied_count}', font=font, fill=(0, 0, 255))
    y_offset += 40  # Espacio entre las estadísticas
    draw.text((start_x, y_offset), f'Plazas libres: {free_count}', font=font, fill=(0, 255, 0))
    y_offset += 40  # Espacio para la tasa de ocupación

    # Listar las plazas libres
    for idx, space in enumerate(free_spaces):
        draw.text((start_x, y_offset), f'Plaza {space}', font=font, fill=(0, 0, 0))
        y_offset += 30  # Separar cada línea

    # Convertir la imagen de nuevo a un array de OpenCV
    new_frame = np.array(pil_image)

    return new_frame

def process_video(video, csv):
    """Procesa un video, analizando plaza por plaza y genera un nuevo video."""

    csv_path = csv

    create_json_from_csv(csv_path)
    
    frame_skip=50
    
    input_video_path = video  # Cambia este por tu video de entrada
    output_video_path = "parking_management_processed.mp4"

    # Cargar coordenadas de las plazas desde el JSON
    with open('rectangles.json', 'r') as file:
        parking_coordinates = json.load(file)  # Contiene las plazas como listas de puntos
    
    model = YOLO("Modelos/visdrone_yolov11s.pt")  # Ruta a tu modelo entrenado

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el archivo de entrada {input_video_path}.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if frame_width == 0 or frame_height == 0:
        print("Error: Dimensiones del video de entrada no válidas.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Cambiar a un codec compatible
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (frame_width + 300, frame_height))  # +300 para el fondo

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    parking_status = parking_coordinates

    try:
        while cap.isOpened() and frame_count < total_frames:
            ret, frame = cap.read()
            if not ret or frame is None:
                print(f"Advertencia: No se pudo leer el frame {frame_count}.")
                break

            if frame_count % frame_skip == 0:
                results = model(frame)
                detections = []
                for result in results:
                    for box in result.boxes.xyxy.cpu().numpy():
                        x1, y1, x2, y2 = box
                        detections.append({'bbox': [x1, y1, x2, y2]})
                parking_status = check_occupancy(detections, parking_coordinates)

            frame = draw_frame_with_parking_status(frame, parking_status)
            if frame is None or frame.size == 0:
                print(f"Error: El frame procesado no es válido.")
                break

            out.write(frame)
            print(f"Frame {frame_count} procesado y escrito al archivo de salida.")

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            frame_count += 1

    except Exception as e:
        print(f"Error inesperado: {e}")

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()

def main():
    with gr.Blocks() as app:
        gr.Markdown("### Subir un CSV y un video para la detección")
        csv_input = gr.File(label="Sube el archivo CSV")
        video_input = gr.File(label="Sube el video")

        process_button = gr.Button("Procesar Video")
        process_button.click(
            process_video,
            inputs=[video_input, csv_input]
        )

    app.launch()

if __name__ == "__main__":
    main()

Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


# Prueba - YOLOv11 con modelo dataset descargado de Roboflow

In [14]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display

display.clear_output()
!yolo checks

Ultralytics 8.3.33 🚀 Python-3.11.7 torch-2.2.2 CPU (Intel Core(TM) i5-1038NG7 2.00GHz)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 351.0/465.6 GB disk)

OS                  macOS-10.16-x86_64-i386-64bit
Environment         Darwin
Python              3.11.7
Install             pip
RAM                 16.00 GB
Disk                351.0/465.6 GB
CPU                 Intel Core(TM) i5-1038NG7 2.00GHz
CPU count           8
GPU                 None
GPU count           None
CUDA                None

numpy               ✅ 1.26.4>=1.23.0
matplotlib          ✅ 3.7.5>=3.3.0
opencv-python       ✅ 4.10.0.84>=4.6.0
pillow              ✅ 10.2.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.2.2>=1.8.0
torchvision         ✅ 0.17.2>=0.9.0
tqdm                ✅ 4.65.0>=4.64.0
psutil              ✅ 5.9.0
py-cpuinfo          ✅ 9.0.0
pandas              ✅ 2.1.4>=1.1.4
seaborn             ✅ 0.12.2>=0.11.0
ultralytics-thop 

In [15]:
from roboflow import Roboflow
rf = Roboflow(api_key="Fvrz6p2dCfxJMTYn31Ht")
project = rf.workspace("jorgealbert").project("parking-lot-mlrab")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [18]:
!yolo task=detect mode=train model=yolo11x.pt data="/Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/Parking-Lot-2/data.yaml" epochs=20 imgsz=640

Ultralytics 8.3.33 🚀 Python-3.11.7 torch-2.2.2 CPU (Intel Core(TM) i5-1038NG7 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolo11x.pt, data=/Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/Parking-Lot-2/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

In [21]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt source=video_prueba.mp4 imgsz=640 conf=0.3 save=True

Ultralytics 8.3.33 🚀 Python-3.11.7 torch-2.2.2 CPU (Intel Core(TM) i5-1038NG7 2.00GHz)
YOLO11x summary (fused): 464 layers, 56,829,334 parameters, 0 gradients, 194.4 GFLOPs

video 1/1 (frame 1/1922) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba.mp4: 384x640 122 car parkeds, 2 clear to parkeds, 742.7ms
video 1/1 (frame 2/1922) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba.mp4: 384x640 121 car parkeds, 2 clear to parkeds, 772.9ms
video 1/1 (frame 3/1922) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba.mp4: 384x640 123 car parkeds, 2 clear to parkeds, 730.8ms
video 1/1 (frame 4/1922) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba.mp4: 384x640 123 car parkeds, 2 clear to parkeds, 758.9ms
video 1/1 (frame 5/1922) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba.mp4: 384x640 124 car parkeds, 2 clear to parkeds, 731.5ms
video 1/1 (frame 6/1922) /Users/jorgealbe

# Pruebas

In [5]:
import json
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont  # Importamos PIL para usar la fuente personalizada

# Cargar coordenadas de las plazas desde el JSON
with open('JSON/bounding_boxes_parking3.json', 'r') as file:
    parking_coordinates = json.load(file)  # Contiene las plazas como listas de puntos

def is_point_inside_polygon(point, polygon):
    """Comprueba si un punto (x, y) está dentro del polígono definido por las plazas."""
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def check_occupancy(detections, parking_coordinates):
    """Comprueba si las detecciones ocupan alguna plaza."""
    for park in parking_coordinates:
        park['occupied'] = False  # Reiniciar el estado de ocupación
        for detection in detections:
            det_x1, det_y1, det_x2, det_y2 = detection['bbox']
            vehicle_center = ((det_x1 + det_x2) / 2, (det_y1 + det_y2) / 2)  # Centro del vehículo

            # Verificar si el centro del vehículo está dentro del polígono de la plaza
            if is_point_inside_polygon(vehicle_center, park['points']):
                park['occupied'] = True
                break
    return parking_coordinates

def draw_frame_with_parking_status(frame, parking_coordinates, background_color=(255, 255, 255), font_path="fuentes/Parkinsans-VariableFont_wght.ttf"):
    """Dibuja el estado del parking en el frame, incluyendo un fondo a la derecha para las plazas libres."""
    free_count = 0
    occupied_count = 0
    free_spaces = []  # Lista de plazas libres

    # Dibujar las plazas numeradas y su estado
    for idx, park in enumerate(parking_coordinates):
        points = np.array(park['points'])
        x1, y1 = points[:, 0].min(), points[:, 1].min()
        x2, y2 = points[:, 0].max(), points[:, 1].max()

        color = (0, 0, 255) if park['occupied'] else (0, 255, 0)  # Rojo para ocupada, verde para libre
        if park['occupied']:
            occupied_count += 1
        else:
            free_count += 1
            free_spaces.append(idx + 1)  # Guardar el número de plaza libre (1-indexado)

        # Dibujar rectángulo
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Borde con grosor de 2

        # Agregar número de plaza en la esquina superior izquierda
        cv2.putText(frame, f'{idx + 1}', (x1 + 5, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Crear el fondo adicional (a la derecha)
    frame_height, frame_width = frame.shape[:2]
    new_width = frame_width + 300  # Añadir 300 píxeles para el fondo
    new_frame = np.ones((frame_height, new_width, 3), dtype=np.uint8) * np.array(background_color, dtype=np.uint8)

    # Copiar el video original al nuevo frame
    new_frame[:, :frame_width] = frame

    # Crear un área para mostrar las estadísticas en la parte superior de la columna
    y_offset = 30  # Espacio para las estadísticas arriba de las plazas libres
    start_x = frame_width + 20  # Comienza justo después del video original

    # Usar PIL para cargar la fuente
    pil_image = Image.fromarray(new_frame)
    draw = ImageDraw.Draw(pil_image)

    try:
        font = ImageFont.truetype(font_path, 20)  # Ajusta el tamaño de la fuente si es necesario
    except IOError:
        print("No se pudo cargar la fuente. Asegúrate de tener el archivo .ttf correctamente ubicado.")
        font = ImageFont.load_default()  # Cargar fuente por defecto si no se encuentra

    # Escribir estadísticas de plazas libres y ocupadas
    draw.text((start_x, y_offset), f'Plazas ocupadas: {occupied_count}', font=font, fill=(0, 0, 255))
    y_offset += 40  # Espacio entre las estadísticas
    draw.text((start_x, y_offset), f'Plazas libres: {free_count}', font=font, fill=(0, 255, 0))
    y_offset += 40  # Espacio para separar las estadísticas del listado

    # Listar las plazas libres
    for idx, space in enumerate(free_spaces):
        draw.text((start_x, y_offset), f'Plaza {space}', font=font, fill=(0, 0, 0))
        y_offset += 30  # Separar cada línea

    # Convertir la imagen de nuevo a un array de OpenCV
    new_frame = np.array(pil_image)

    return new_frame

def process_video(input_video_path, output_video_path, parking_coordinates, frame_skip=100):
    """Procesa un video, analizando plaza por plaza y genera un nuevo video."""
    # Cargar el modelo YOLO
    model = YOLO("Modelos/visdrone_yolov11s.pt")  # Ruta a tu modelo entrenado

    # Abrir el video de entrada
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Codec para el video de salida
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    frame_count = 0  # Contador de frames
    parking_status = parking_coordinates  # Estado inicial de las plazas
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total de frames en el video

    try:
        while cap.isOpened() and frame_count < total_frames:
            ret, frame = cap.read()
            if not ret:
                break

            # Procesar solo cada `frame_skip` frames
            if frame_count % frame_skip == 0:
                results = model(frame)
                detections = []
                for result in results:
                    for box in result.boxes.xyxy.cpu().numpy():  # Extraer coordenadas del bounding box
                        x1, y1, x2, y2 = box
                        detections.append({'bbox': [x1, y1, x2, y2]})

                # Actualizar el estado de las plazas
                parking_status = check_occupancy(detections, parking_coordinates)

            # Dibujar el estado más reciente en el frame
            frame = draw_frame_with_parking_status(frame, parking_status)

            # Escribir el frame procesado en el video de salida
            out.write(frame)

            # Mostrar el frame en tiempo real (opcional, desactívalo para evitar bloqueos)
            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir si se presiona 'q'
                break
                

            # Incrementar el contador de frames
            frame_count += 1

    except Exception as e:
        print(f"Error inesperado: {e}")

    finally:
        # Liberar recursos
        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Rutas de los archivos
input_video_path = "Videos/video_parking3.mp4"  # Cambia este por tu video de entrada
output_video_path = "parking_management_processed.mp4"

# Ruta de la fuente personalizada
font_path = "fuentes/Parkinsans-VariableFont_wght.ttf"  # Ajusta esta ruta si es necesario

# Procesar el video
process_video(input_video_path, output_video_path, parking_coordinates, frame_skip=100)


0: 448x640 1 pedestrian, 56 cars, 3 vans, 2 trucks, 203.1ms
Speed: 4.1ms preprocess, 203.1ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)


OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match 

OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.


0: 448x640 55 cars, 3 vans, 1 truck, 160.7ms
Speed: 3.5ms preprocess, 160.7ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)


OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match 

...

OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.


0: 448x640 53 cars, 2 vans, 2 trucks, 164.0ms
Speed: 3.3ms preprocess, 164.0ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)


OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match 


0: 448x640 3 pedestrians, 55 cars, 3 vans, 1 truck, 2 motors, 993.2ms
Speed: 3.2ms preprocess, 993.2ms inference, 23.6ms postprocess per image at shape (1, 3, 448, 640)


OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match video size.
OpenCV: Frame size does not match 

In [2]:
# FUNCIONA EL GUARDADO
from ultralytics import YOLO

# Archivo de entrada y salida
input_video = "video_prueba_recortado.mp4"
output_video = "parking_management_processed.mp4"

# Cargar el modelo entrenado
model = YOLO("visdrone_yolov11s.pt")  # Cambia la ruta al modelo entrenado

# Procesar el video de entrada y guardar la salida
results = model.predict(source=input_video, save=True, save_txt=False, conf=0.3, imgsz=640)

# Renombrar el archivo de salida generado por YOLO para mantener consistencia
import os
os.rename("runs/detect/predict/video_prueba_recortado.mp4", output_video)

print(f"Video procesado y guardado correctamente en: {output_video}")




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1923) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba_recortado.mp4: 576x640 33 cars, 4 vans, 2 trucks, 2 buss, 245.4ms
video 1/1 (frame 2/1923) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba_recortado.mp4: 576x640 33 cars, 4 vans, 2 trucks, 2 buss, 252.0ms
video 1/1 (frame 3/1923) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba_recortado.mp4: 576x640 32 c

2024-11-23 18:51:13.308 python[59148:1117159] WARNING: -finishWriting should not be called on the main thread.


FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/predict/video_prueba_recortado.mp4' -> 'parking_management_processed.mp4'

In [4]:
from ultralytics import YOLO
import json
import os

# Archivos de entrada y salida
input_video = "video_prueba_recortado.mp4"
output_folder = "runs/detect/predict/"
output_video = os.path.join(output_folder, "parking_management_processed.mp4")
json_file = "bounding_boxes.json"  # Archivo con las coordenadas de las plazas de parking

# Asegurarse de que la carpeta de salida exista
os.makedirs(output_folder, exist_ok=True)

# Cargar el modelo YOLO
model = YOLO("visdrone_yolov11s.pt")  # Cambia la ruta al modelo entrenado

# Leer coordenadas de plazas de estacionamiento desde el archivo JSON
with open(json_file, "r") as f:
    parking_data = json.load(f)

# Convertir las coordenadas en una lista de polígonos
parking_polygons = []
for parking_space in parking_data:
    polygon = parking_space["points"]  # Asumimos que "points" es una lista de [(x1, y1), (x2, y2), ...]
    parking_polygons.append(polygon)

# Verificar si un punto está dentro de un polígono
def is_inside_polygon(point, polygon):
    """Comprueba si un punto está dentro de un polígono (algoritmo de ray casting)."""
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

# Inicializar variables de reporte
frame_reports = []  # Guardará el reporte por cada frame (ocupados/libres)

# Procesar el video con YOLO (sin modificar directamente los frames)
results = model.predict(source=input_video, save=True, stream=True, conf=0.3, imgsz=640)

for idx, result in enumerate(results):
    # Obtener detecciones de cada frame
    detections = result.boxes.xyxy.cpu().numpy()  # Bounding boxes [x1, y1, x2, y2]
    labels = result.boxes.cls.cpu().numpy()       # Clases detectadas

    # Contar ocupación por frame
    frame_occupied = set()

    for i, parking_space in enumerate(parking_polygons):
        for box in detections:
            x1, y1, x2, y2 = box
            vehicle_center = ((x1 + x2) / 2, (y1 + y2) / 2)  # Centro del bounding box

            # Verificar si el centro del vehículo está dentro del polígono
            if is_inside_polygon(vehicle_center, parking_space):
                frame_occupied.add(i)
                break

    # Contar plazas ocupadas y libres
    occupied_spaces = len(frame_occupied)
    free_spaces = len(parking_polygons) - occupied_spaces

    # Guardar información del frame
    frame_reports.append({
        "frame_index": idx,  # Usamos el índice del ciclo como el índice del frame
        "occupied_spaces": occupied_spaces,
        "free_spaces": free_spaces
    })

    print(f"Frame {idx}: {occupied_spaces} ocupados, {free_spaces} libres.")

# Guardar reporte en un archivo JSON para análisis posterior
report_file = os.path.join(output_folder, "parking_report.json")
with open(report_file, "w") as f:
    json.dump(frame_reports, f, indent=4)

# Confirmar que el archivo de salida existe
if os.path.exists(output_video):
    print(f"Video procesado y guardado correctamente en: {output_video}")
    print(f"Reporte de ocupación guardado en: {report_file}")
else:
    print(f"Hubo un problema al guardar el video en: {output_video}")

2024-11-23 19:19:18.830 python[59488:1141236] WARNING: -finishWriting should not be called on the main thread.


video 1/1 (frame 1/1923) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba_recortado.mp4: 576x640 33 cars, 4 vans, 2 trucks, 2 buss, 209.9ms
Frame 0: 31 ocupados, 3 libres.
video 1/1 (frame 2/1923) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba_recortado.mp4: 576x640 33 cars, 4 vans, 2 trucks, 2 buss, 226.2ms
Frame 1: 31 ocupados, 3 libres.
video 1/1 (frame 3/1923) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba_recortado.mp4: 576x640 32 cars, 3 vans, 2 trucks, 2 buss, 206.7ms
Frame 2: 31 ocupados, 3 libres.
video 1/1 (frame 4/1923) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba_recortado.mp4: 576x640 1 pedestrian, 31 cars, 3 vans, 2 trucks, 2 buss, 228.1ms
Frame 3: 32 ocupados, 2 libres.
video 1/1 (frame 5/1923) /Users/jorgealbert/Desktop/IMMUNE/AÑO 3/IA/PF - Parking Lot/video_prueba_recortado.mp4: 576x640 1 pedestrian, 31 cars, 3 vans, 3 trucks, 2 buss, 202.8ms
Frame 4: 32 ocupado

2024-11-23 19:27:22.842 python[59488:1141236] WARNING: -finishWriting should not be called on the main thread.


In [ ]:
import cv2

from ultralytics import solutions

# Video capture
cap = cv2.VideoCapture("video_prueba_recortado.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
#video_writer = cv2.VideoWriter("parking management.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize parking management object
parking_manager = solutions.ParkingManagement(
    model="yolov8n.pt",  # path to model file
    json_file="bounding_boxes.json",  # path to parking annotations file
)

while cap.isOpened():
    ret, im0 = cap.read()
    if not ret:
        break
    im0 = parking_manager.process_data(im0)
    cv2.imshow("parking", im0)
    cv2.waitKey(0)
#    video_writer.write(im0)

cap.release()
#video_writer.release()
cv2.destroyAllWindows()

Ultralytics Solutions: ✅ {'region': None, 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': 'bounding_boxes.json', 'model': 'yolov8n.pt'}

0: 576x640 1 potted plant, 21 cell phones, 119.8ms
Speed: 5.8ms preprocess, 119.8ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 1 potted plant, 21 cell phones, 94.8ms
Speed: 2.4ms preprocess, 94.8ms inference, 1.1ms postprocess per image at shape (1, 3, 576, 640)


2024-11-20 13:26:50.397 python[39260:445216] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-20 13:26:50.397 python[39260:445216] +[IMKInputSession subclass]: chose IMKInputSession_Legacy



0: 576x640 1 bus, 1 potted plant, 21 cell phones, 100.4ms
Speed: 5.3ms preprocess, 100.4ms inference, 0.8ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 1 bus, 1 potted plant, 21 cell phones, 101.0ms
Speed: 6.0ms preprocess, 101.0ms inference, 0.9ms postprocess per image at shape (1, 3, 576, 640)


In [ ]:
import cv2

imagen=cv2.imread("banana.jpg")

if imagen is None:
    print(".....")
else:
    print("entra")
    cv2.imshow("x", imagen)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

entra


2024-11-20 13:25:36.423 python[39247:443260] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-20 13:25:36.423 python[39247:443260] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
